# Use Consensus Labels as Multiple-Sequence-Alignment (MSA)

In this notebook, we will use `mdciao` and the consensus labels of the [GPCRdb](https://gpcrdb.org) to structurally align four GPCR structures of four receptors with low sequence identity:

* opsin, `OPS` in the rest of the notebook
* beta2 adrenergic receptor, `B2AR` in the rest of the notebook
* mu-opioid receptor, `MUOR` in the rest of the notebook
* dopamine D1 receptor, `DOP` in the rest of the notebook

Whereas we use directly PDB structures, the point should be clear that the MSA works on any arbitary geometries and topologies that can be imported into the notebook, **in particular user provided trajectories**.

In [1]:
import mdciao
import nglview
import matplotlib

# Load PDB  Structures into the Notebook

In [2]:
pdbs = {"OPS"  : mdciao.cli.pdb("3CAP"), 
        "B2AR" : mdciao.cli.pdb("3SN6"), 
        "MUOR" : mdciao.cli.pdb("6DDF"),
        "DOP"  : mdciao.cli.pdb("7CKW")}

Checking https://files.rcsb.org/download/3CAP.pdb ...Please cite the following 3rd party publication:
 * Crystal structure of the ligand-free G-protein-coupled receptor opsin
   Park, J.H. et al., Nature 2008
   https://doi.org/10.1038/nature07063
Checking https://files.rcsb.org/download/3SN6.pdb ...Please cite the following 3rd party publication:
 * Crystal structure of the beta2 adrenergic receptor-Gs protein complex
   Rasmussen, S.G. et al., Nature 2011
   https://doi.org/10.1038/nature10361
Checking https://files.rcsb.org/download/6DDF.pdb ...

/home/guille/miniconda3/lib/python3.11/site-packages/mdtraj/formats/pdb/pdbfile.py:206: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn(


Please cite the following 3rd party publication:
 * Structure of the mu-opioid receptor-Giprotein complex.
   Koehl, A. et al., Nature 2018
   https://doi.org/10.1038/s41586-018-0219-7
Checking https://files.rcsb.org/download/7CKW.pdb ...Please cite the following 3rd party publication:
 * Ligand recognition and allosteric regulation of DRD1-Gs signaling complexes.
   Xiao, P. et al., Cell 2021
   https://doi.org/10.1016/j.cell.2021.01.028


# Load Consensus Labels from the GPCRdb into the Notebook

In [3]:
maps = { "OPS": mdciao.nomenclature.LabelerGPCR("opsd_bovin"),
        "B2AR": mdciao.nomenclature.LabelerGPCR("adrb2_human"),
        "MUOR": mdciao.nomenclature.LabelerGPCR("oprm_mouse"), 
        "DOP" : mdciao.nomenclature.LabelerGPCR("DRD1_HUMAN")
       }

No local file ./opsd_bovin.xlsx found, checking online in
https://gpcrdb.org/services/residues/extended/opsd_bovin ...done!
Please cite the following reference to the GPCRdb:
 * Kooistra et al, (2021) GPCRdb in 2021: Integrating GPCR sequence, structure and function
   Nucleic Acids Research 49, D335--D343
   https://doi.org/10.1093/nar/gkaa1080
For more information, call mdciao.nomenclature.references()
No local file ./adrb2_human.xlsx found, checking online in
https://gpcrdb.org/services/residues/extended/adrb2_human ...

/home/guille/Programs/mdciao/mdciao/nomenclature/nomenclature.py:386: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = _read_json(a.text)


done!
Please cite the following reference to the GPCRdb:
 * Kooistra et al, (2021) GPCRdb in 2021: Integrating GPCR sequence, structure and function
   Nucleic Acids Research 49, D335--D343
   https://doi.org/10.1093/nar/gkaa1080
For more information, call mdciao.nomenclature.references()
No local file ./oprm_mouse.xlsx found, checking online in
https://gpcrdb.org/services/residues/extended/oprm_mouse ...

/home/guille/Programs/mdciao/mdciao/nomenclature/nomenclature.py:386: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = _read_json(a.text)


done!
Please cite the following reference to the GPCRdb:
 * Kooistra et al, (2021) GPCRdb in 2021: Integrating GPCR sequence, structure and function
   Nucleic Acids Research 49, D335--D343
   https://doi.org/10.1093/nar/gkaa1080
For more information, call mdciao.nomenclature.references()
No local file ./DRD1_HUMAN.xlsx found, checking online in
https://gpcrdb.org/services/residues/extended/drd1_human ...

/home/guille/Programs/mdciao/mdciao/nomenclature/nomenclature.py:386: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = _read_json(a.text)


done!
Please cite the following reference to the GPCRdb:
 * Kooistra et al, (2021) GPCRdb in 2021: Integrating GPCR sequence, structure and function
   Nucleic Acids Research 49, D335--D343
   https://doi.org/10.1093/nar/gkaa1080
For more information, call mdciao.nomenclature.references()


/home/guille/Programs/mdciao/mdciao/nomenclature/nomenclature.py:386: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = _read_json(a.text)


# Use the Consensus Labels to Trim down the PDBs to the just Receptors
This can happen regardless of chain definitions and co-crystalized entities

In [4]:
pdb_just_receptor = {}
for key, pdb  in pdbs.items():
    print(key)
    receptor_residue_idxs = mdciao.nomenclature.guess_by_nomenclature(maps[key], 
                                                               pdb.top, 
                                                               fragments="resSeq",
                                                               return_residue_idxs=True,
                                                               accept_guess=True,
                                                              return_str=False)
    pdb_just_receptor[key] = mdciao.fragments.fragment_slice(pdb, [receptor_residue_idxs])
    print()

OPS
Auto-detected fragments with method 'resSeq'
fragment      0 with    326 AAs     MET1 (   0) -   ASN326 (325 ) (0) 
fragment      1 with    326 AAs     MET1 ( 326) -   ASN326 (651 ) (1) 
fragment      2 with      4 AAs     NAG1 ( 652) -     BMA4 (655 ) (2) 
fragment      3 with      6 AAs     NAG1 ( 656) -     BMA4 (661 ) (3)  resSeq jumps
fragment      4 with      2 AAs     NAG1 ( 662) -     NAG2 (663 ) (4) 
fragment      5 with      4 AAs   BGL801 ( 664) -   BGL804 (667 ) (5) 
fragment      6 with      1 AAs   PLM901 ( 668) -   PLM901 (668 ) (6) 
fragment      7 with      2 AAs   BGL805 ( 669) -   BGL806 (670 ) (7) 
fragment      8 with      1 AAs   PLM902 ( 671) -   PLM902 (671 ) (8) 
fragment      9 with     10 AAs   HOH902 ( 672) -   HOH907 (681 ) (9)  resSeq jumps
The GPCR-labels align best with fragments: [0] (first-last: MET1-ASN326).

B2AR
Auto-detected fragments with method 'resSeq'
fragment      0 with     51 AAs     THR9 (   0) -    GLN59 (50  ) (0) 
fragment      1 wit

# Receptors are not 3D Aligned
This is a bit obvious because the come from different PDBs, but helps highlight the point

In [5]:
colors = {"MUOR":"tab:red", 
          "OPS":"tab:blue", 
          "B2AR":"tab:green",
          "DOP": "tab:orange"}
iwd = nglview.NGLWidget()
for ii, (key, geom) in enumerate(pdb_just_receptor.items()):
    iwd.add_trajectory(geom)
    iwd.clear_representations(component=ii)
    iwd.add_cartoon(color=matplotlib.colors.to_hex(colors[key]), component=ii)
iwd

NGLWidget()

# Use the Consensus Labels to generate an [AlignerConsensus](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.nomenclature.AlignerConsensus.html#mdciao.nomenclature.AlignerConsensus) for MSA

In [6]:
AC = mdciao.nomenclature.AlignerConsensus(maps,
                                          tops={key : geom.top for key, geom in pdb_just_receptor.items()})

/home/guille/Programs/mdciao/mdciao/nomenclature/nomenclature.py:2369: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  labs_out = [subset[oo] for oo in order]


# Sequence Identity within Residues with Consensus Labels

In [7]:
AC.sequence_match()

/home/guille/Programs/mdciao/mdciao/nomenclature/nomenclature.py:1829: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: "%u%%" % x)


,OPS,B2AR,MUOR,DOP
OPS,100%,22%,24%,21%
B2AR,22%,100%,28%,42%
MUOR,24%,28%,100%,28%
DOP,21%,42%,28%,100%


# Pick a Reference Structure (e.g. Opsin) and 3D-align all Receptors on It
We do this using the [CAidxs_match](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.nomenclature.AlignerConsensus.html#mdciao.nomenclature.AlignerConsensus.CAidxs_match) method of the [AlignerConsensus](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.nomenclature.AlignerConsensus.html#mdciao.nomenclature.AlignerConsensus) that will generate pairs of indices matching one another via their consensus labels. For brevity, here we show the example in the "3.50...3.59" region of TM3, but for the 3D alignment, we take all consensus labels

In [8]:
AC.CAidxs_match("3.5*", keys=["OPS","B2AR"])

,consensus,OPS,B2AR
115,3.50x50,1065,746
116,3.51x51,1076,757
117,3.52x52,1088,769
118,3.53x53,1095,780
119,3.54x54,1102,785
120,3.55x55,1109,793
121,3.56x56,1115,800


In [9]:
ref_key = "OPS"
ref_geom = pdb_just_receptor[ref_key]
for key, geom in pdb_just_receptor.items():
     if key!=ref_key:
        ref_CAs, key_CAs = AC.CAidxs_match(keys=[ref_key, key])[[ref_key, key]].values.T
        geom.superpose(ref_geom, atom_indices=key_CAs, ref_atom_indices=ref_CAs)


# Receptors are now 3D-aligned

In [10]:
iwd = nglview.NGLWidget()
for ii, (key, geom) in enumerate(pdb_just_receptor.items()):
    iwd.add_trajectory(geom)
    iwd.clear_representations(component=ii)
    iwd.add_cartoon(color=matplotlib.colors.to_hex(colors[key]), component=ii)
iwd

NGLWidget()